# Q2

## Pararmeters Calculation

In [2]:
import numpy as np
import pandas as pd

In [145]:
import time

In [146]:
sheet_1 = pd.read_csv("1.csv",encoding='gb18030')

/Users/duli/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [147]:
sheet_2 = pd.read_csv("2.csv",encoding='gb18030')

/Users/duli/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [148]:
excel3 = pd.ExcelFile('3.xlsx')
sheet_3 = pd.read_excel(excel3)

In [149]:
excel4 = pd.ExcelFile('4.xlsx')
sheet_4 = pd.read_excel(excel4)

In [150]:
sheet_1['create_dt'] = sheet_1['create_dt'].astype('datetime64')

In [151]:
sheet_2['create_dt'] = sheet_2['create_dt'].astype('datetime64')

In [152]:
sheet_1 = sheet_1[sheet_1['is_finished']==1]

In [153]:
sheet_2 = sheet_2[sheet_2['is_finished']==1]

In [14]:
RCDS_Weight = np.array([(1, 1/7, 1/5),(7, 1, 2),(5, 1/2, 1)])

In [15]:
W_ = {}
W_sum = 0
for i in range(RCDS_Weight.shape[0]):
    name_matrix = ['Wc_', 'Wd_', 'Ws_']
    temp = 1
    for j in range(RCDS_Weight.shape[1]):
        temp = RCDS_Weight[i, j]*temp
    W_[name_matrix[i]] = np.power(temp,1/3)
    W_sum += W_[name_matrix[i]]
    
    

In [16]:
for i in W_:
    W_[i] = round(W_[i]/W_sum,3)
    

In [17]:
W_sum

4.0730617812055625

In [18]:
W_

{'Wc_': 0.075, 'Wd_': 0.592, 'Ws_': 0.333}

In [8]:
b = np.linalg.eigvals(RCDS_Weight)

In [9]:
lamb = b[0].real

In [10]:
lamb

3.014151882186208

In [11]:
CI = (lamb - 3)/2

In [12]:
CI

0.0070759410931040545

In [13]:
RCDS_Weight

array([[1.        , 0.14285714, 0.2       ],
       [7.        , 1.        , 2.        ],
       [5.        , 0.5       , 1.        ]])

In [165]:
RI = 0.58

In [166]:
CR = CI / RI

In [167]:
CR

0.012199898436385536

## Implementation

In [168]:
a = sheet_3[np.logical_and((sheet_3['begin_time']<=np.datetime64('2017-09-02 23:59:59')),(sheet_3['end_time']>=np.datetime64('2017-09-02 00:00:00')))]

In [169]:
sheet_3_vaild = sheet_3[sheet_3['state']==5]

In [170]:
Q2_start_time = np.datetime64('2016-11-30 00:00:00')

In [171]:
Q2_days = np.datetime64('2016-11-30')

In [172]:
Q2_day_time = np.datetime64('2016-11-30 23:59:59')

In [173]:
Q2_end_time = np.datetime64('2019-01-02 23:59:59')

In [174]:
Discount_info_per_day = {}

In [175]:
while Q2_start_time<=Q2_end_time:
    
    a = sheet_3_vaild[np.logical_and((sheet_3_vaild['begin_time']<=Q2_day_time),(sheet_3_vaild['end_time']>=Q2_start_time))]
    Discount_info_per_day[Q2_days] = a
    
    Q2_start_time = Q2_start_time + 86400
    Q2_days = Q2_days + 1
    Q2_day_time = Q2_day_time + 86400

## After 2017-03-07

In [176]:
C_per_day = {}

In [177]:
S_per_day = {}

In [178]:
D_per_day ={}

In [179]:
Q2_end_time = np.datetime64('2019-01-02')

In [180]:
Q2_start_time = np.datetime64('2017-03-07')

In [181]:
sheet_1 = pd.concat([sheet_1,sheet_2])

In [182]:
while Q2_start_time<=Q2_end_time:
    Discount_record = sheet_1[sheet_1['create_dt']==Q2_start_time]
    test_Q2= pd.concat([Discount_record['sku_id'],Discount_info_per_day[Q2_start_time]['sku_id']])
    Class_of_goods = len(test_Q2.unique())
    C_per_day[Q2_start_time] = Class_of_goods
    Sales_count_sheet = Discount_record
    Sales_count = Discount_record['sku_cnt'].sum()
    S_per_day[Q2_start_time] = Sales_count
    Dis_info_board= Discount_info_per_day[Q2_start_time]
    Dis_info_un = Dis_info_board.groupby(['sku_id']).sum()
    Dis_info_un['id'] = (Dis_info_un['pdj_price']-Dis_info_un['promotion_price'])/Dis_info_un['pdj_price']
    Dis_rate_board = (Dis_info_un['id'].sum())/len(Dis_info_un)
    for i in range(len(Dis_info_un)):
        Discount_record = Discount_record[Discount_record['sku_id']!=Dis_info_un.index[1]]
    
    Discount_record_un = Discount_record.groupby(['sku_id']).sum()
    Discount_record_un['is_finished'] = (Discount_record_un['sku_prc'] - Discount_record_un['sku_sale_prc'])/Discount_record_un['sku_prc']
    if Discount_record_un['is_finished'].sum()!=0:
        Dis_rate_record = (Discount_record_un['is_finished'].sum())/len(Discount_record_un)
    else:
        Dis_rate_record = 0
    Dis_rate = (Dis_rate_board + Dis_rate_record)/2
    D_per_day[Q2_start_time] = Dis_rate
    
    Q2_start_time = Q2_start_time + 1
        
    

## Before 2017-03-07

In [183]:
Q2_end_time = np.datetime64('2017-03-06')

In [184]:
Q2_start_time = np.datetime64('2016-11-30')

In [191]:
while Q2_start_time<=Q2_end_time:
    Discount_record = sheet_1[sheet_1['create_dt']==Q2_start_time]
    No_of_Goods = len(Discount_record['sku_id'].unique())
    C_per_day[Q2_start_time] = No_of_Goods
    Sales_count_bef = Discount_record['sku_cnt'].sum()
    S_per_day[Q2_start_time] = Sales_count_bef
    Discount_un = Discount_record.groupby(['sku_id']).sum()
    Discount_un['is_finished'] = (Discount_un['sku_prc'] - Discount_un['sku_sale_prc'])/Discount_un['sku_prc']
    
    if Discount_un['is_finished'].sum()!=0:
        Dis_rate = (Discount_un['is_finished'].sum())/len(Discount_un)
    else:
        Dis_rate = 0

    D_per_day[Q2_start_time] = Dis_rate
    
    Q2_start_time = Q2_start_time + 1

In [213]:

start_time = np.datetime64('2016-11-30')
min_c = 10000
for i in range(len(C_per_day)):
    if (C_per_day[start_time]<min_c):
        min_c = C_per_day[start_time]
    start_time = start_time + 1


In [214]:

start_time = np.datetime64('2016-11-30')
max_c = 10
for i in range(len(C_per_day)):
    if (C_per_day[start_time]>max_c):
        max_c = C_per_day[start_time]
    start_time = start_time + 1


In [217]:
start_time = np.datetime64('2016-11-30')
min_s = 10000
for i in range(len(S_per_day)):
    if (S_per_day[start_time]<min_s):
        min_s = S_per_day[start_time]
    start_time = start_time + 1


In [219]:

start_time = np.datetime64('2016-11-30')
max_s = 10
for i in range(len(S_per_day)):
    if (S_per_day[start_time]>max_s):
        max_s = S_per_day[start_time]
    start_time = start_time + 1



In [222]:
start_time = np.datetime64('2016-11-30')
min_d = 10000
for i in range(len(D_per_day)):
    if (D_per_day[start_time]<min_d):
        min_d = D_per_day[start_time]
    start_time = start_time + 1



In [244]:

start_time = np.datetime64('2016-11-30')
max_d = 0.1
for i in range(len(D_per_day)):
    if (D_per_day[start_time]>max_d):
        max_d = D_per_day[start_time]
    start_time = start_time + 1



In [226]:
min_c

110

In [227]:
max_c

3056

In [228]:
min_d

0.029546289438748687

In [245]:
max_d

0.22859252190408413

In [230]:
min_s

0

In [231]:
max_s

19590

## Calculating the discount rate


In [236]:
value_c = {}
start_time = np.datetime64('2016-11-30')
for i in range(len(C_per_day)):
    value_c[start_time] = np.ceil(C_per_day[start_time]*5/(max_c-min_c))
    start_time = start_time + 1

In [246]:
value_d = {}
start_time = np.datetime64('2016-11-30')
for i in range(len(D_per_day)):
    value_d[start_time] = np.ceil((D_per_day[start_time]-min_d)*5/(max_d-min_d))
    start_time = start_time + 1

In [242]:
value_s = {}
start_time = np.datetime64('2016-11-30')
for i in range(len(S_per_day)):
    value_s[start_time] = np.ceil(S_per_day[start_time]*5/(max_s-min_s))
    start_time = start_time + 1

In [248]:
Value_dic = {}
start_time = np.datetime64('2016-11-30')

for i in range(len(value_c)):
    value_temp = value_c[start_time]*W_['Wc_']+value_d[start_time]*W_['Wd_']+value_s[start_time]*W_['Ws_']
    Value_dic[start_time] = value_temp
    
    start_time = start_time + 1
    

In [258]:
value_c_ = {}
start_time = np.datetime64('2016-11-30')
for i in range(len(C_per_day)):
    value_c_[start_time] = (C_per_day[start_time]/(max_c-min_c))
    start_time = start_time + 1

In [259]:
value_d_ = {}
start_time = np.datetime64('2016-11-30')
for i in range(len(D_per_day)):
    value_d_[start_time] = ((D_per_day[start_time]-min_d)/(max_d-min_d))
    start_time = start_time + 1

In [260]:
value_s_ = {}
start_time = np.datetime64('2016-11-30')
for i in range(len(S_per_day)):
    value_s_[start_time] = (S_per_day[start_time]/(max_s-min_s))
    start_time = start_time + 1

In [261]:
Value_dic_ = {}
start_time = np.datetime64('2016-11-30')

for i in range(len(value_c_)):
    value_temp = value_c_[start_time]*W_['Wc_']+value_d_[start_time]*W_['Wd_']+value_s_[start_time]*W_['Ws_']
    Value_dic_[start_time] = value_temp
    
    start_time = start_time + 1
    

{numpy.datetime64('2016-11-30'): 0.2265308548496164,
 numpy.datetime64('2016-12-01'): 0.1678058944465329,
 numpy.datetime64('2016-12-02'): 0.1712594551890287,
 numpy.datetime64('2016-12-03'): 0.18572752376847146,
 numpy.datetime64('2016-12-04'): 0.15145026739192266,
 numpy.datetime64('2016-12-05'): 0.13829067474634532,
 numpy.datetime64('2016-12-06'): 0.2589102366565755,
 numpy.datetime64('2016-12-07'): 0.15652731730531783,
 numpy.datetime64('2016-12-08'): 0.21482998415871826,
 numpy.datetime64('2016-12-09'): 0.22230154983323588,
 numpy.datetime64('2016-12-10'): 0.16772563281210706,
 numpy.datetime64('2016-12-11'): 0.15111340418881777,
 numpy.datetime64('2016-12-12'): 0.1848742378913406,
 numpy.datetime64('2016-12-13'): 0.16605008487043255,
 numpy.datetime64('2016-12-14'): 0.1569959776224823,
 numpy.datetime64('2016-12-15'): 0.16162714748059975,
 numpy.datetime64('2016-12-16'): 0.18570276065789634,
 numpy.datetime64('2016-12-17'): 0.16789736743218653,
 numpy.datetime64('2016-12-18'): 0